In [1]:
%pip install -e package_calculate_income

Obtaining file:///Users/mateusz/Desktop/final-assignment/package_calculate_income
  Preparing metadata (setup.py) ... done
  Attempting uninstall: package-calculate-income
    Found existing installation: package-calculate-income 0.0.1
    Uninstalling package-calculate-income-0.0.1:
      Successfully uninstalled package-calculate-income-0.0.1
  Running setup.py develop for package-calculate-income
Note: you may need to restart the kernel to use updated packages.


In [2]:
import configparser
import os
import pandas as pd
import numpy as np
import package_calculate_income.calculate_income_jst.jst_income_calc as jst_calc
import package_calculate_income.calculate_income_stat.mean_income_calc as mean_calc
import package_calculate_income.calculate_income_stat.variance_income_calc as var_calc

In [3]:
dir_name = os.path.dirname("script.ipynb")
config = configparser.ConfigParser()
config.read(os.path.join(dir_name, "config.ini"))

['config.ini']

In [5]:
df_gm = jst_calc.income_comparison(config["data"]["y1_f1"], config["data"]["y2_f1"])
df_pow = jst_calc.income_comparison(config["data"]["y1_f2"], config["data"]["y2_f2"])
df_mnpp = jst_calc.income_comparison(config["data"]["y1_f3"], config["data"]["y2_f3"])
df_metrop = jst_calc.income_comparison(config["data"]["y1_f4"], config["data"]["y2_f4"])
df_woj = jst_calc.income_comparison(config["data"]["y1_f5"], config["data"]["y2_f5"])
df_gm

,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku
kod,,,,,,,,
020000,DOLNOŚLĄSKIE,02,00,00,wojewodztwo,8.986697e+09,9.157574e+09,-170876937.5
040000,KUJAWSKO-POMORSKIE,04,00,00,wojewodztwo,4.871153e+09,5.007150e+09,-135997375.0
060000,LUBELSKIE,06,00,00,wojewodztwo,4.076763e+09,4.201244e+09,-124480562.5
080000,LUBUSKIE,08,00,00,wojewodztwo,2.430983e+09,2.519327e+09,-88344062.5
100000,ŁÓDZKIE,10,00,00,wojewodztwo,6.682968e+09,6.837137e+09,-154169062.5
120000,MAŁOPOLSKIE,12,00,00,wojewodztwo,9.673090e+09,9.759448e+09,-86357875.0
140000,MAZOWIECKIE,14,00,00,wojewodztwo,2.376420e+10,2.437245e+10,-608248937.5
160000,OPOLSKIE,16,00,00,wojewodztwo,2.261440e+09,2.322817e+09,-61376937.5
180000,PODKARPACKIE,18,00,00,wojewodztwo,4.105140e+09,4.196095e+09,-90954750.0


In [6]:
df_pow

,nazwa-JST,WK,PK,GK,rodzaj-JST,dochod-2020,dochod-2019,roznica-wzgl-poprz-roku
kod,,,,,,,,
020100,BOLESŁAWIECKI,02,01,00,powiat,2.075759e+08,2.144523e+08,-6.876400e+06
020200,DZIERŻONIOWSKI,02,02,00,powiat,2.080848e+08,2.136905e+08,-5.605659e+06
020300,GŁOGOWSKI,02,03,00,powiat,2.821103e+08,2.960282e+08,-1.391783e+07
020400,GÓROWSKI,02,04,00,powiat,5.543528e+07,5.642302e+07,-9.877463e+05
020500,JAWORSKI,02,05,00,powiat,1.019589e+08,1.029109e+08,-9.520390e+05
...,...,...,...,...,...,...,...,...
321400,STARGARDZKI,32,14,00,powiat,2.712376e+08,2.721161e+08,-8.784585e+05
321500,SZCZECINECKI,32,15,00,powiat,1.490473e+08,1.551099e+08,-6.062644e+06
321600,ŚWIDWIŃSKI,32,16,00,powiat,8.430540e+07,8.769798e+07,-3.392576e+06


In [ ]:
df_mnpp
